In [1]:
import os
from datetime import datetime
import requests as r
import numpy as np
import pandas as pd
import geopandas as gp
import io
import matplotlib.pyplot as plt
#from osgeo import gdal
import rasterio as rio
from rasterio.mask import mask
from rasterio.enums import Resampling
from rasterio.shutil import copy
from rasterio.transform import from_origin
import pyproj
from pyproj import Proj
from shapely.ops import transform
import xarray as xr
#import geoviews as gv
from cartopy import crs
import hvplot.xarray
import holoviews as hv
import boto3
from rasterio.session import AWSSession
import rioxarray as rxr
from rioxarray.merge import merge_arrays
import requests
import h5py
from satpy import Scene
#from satpy.readers import hdf5_reader

In [2]:
# AUTHENTICATION CONFIGURATION
from netrc import netrc
from subprocess import Popen, DEVNULL, STDOUT
from getpass import getpass
from sys import platform

urs = 'urs.earthdata.nasa.gov'    # Earthdata URL to call for authentication
prompts = ['Enter NASA Earthdata Login Username \n(or create an account at urs.earthdata.nasa.gov): ',
           'Enter NASA Earthdata Login Password: ']

# Determine if netrc file exists, and if it includes NASA Earthdata Login Credentials
if 'win' in platform:
    nrc = '_netrc'
else:
    nrc = '.netrc'
try:
    netrcDir = os.path.expanduser(f"~/{nrc}")
    netrc(netrcDir).authenticators(urs)[0]
    del netrcDir

# If not, create a netrc file and prompt user for NASA Earthdata Login Username/Password
except FileNotFoundError:
    homeDir = os.path.expanduser("~")

    # Windows OS won't read the netrc unless this is set
    Popen(f'setx HOME {homeDir}', shell=True, stdout=DEVNULL);

    if nrc == '.netrc':
        Popen(f'touch {homeDir + os.sep}{nrc} | chmod og-rw {homeDir + os.sep}{nrc}', shell=True, stdout=DEVNULL, stderr=STDOUT);

    # Unable to use touch/chmod on Windows OS
    Popen(f'echo machine {urs} >> {homeDir + os.sep}{nrc}', shell=True)
    Popen(f'echo login {getpass(prompt=prompts[0])} >> {homeDir + os.sep}{nrc}', shell=True)
    Popen(f'echo password {getpass(prompt=prompts[1])} >> {homeDir + os.sep}{nrc}', shell=True)
    del homeDir

# Determine OS and edit netrc file if it exists but is not set up for NASA Earthdata Login
except TypeError:
    homeDir = os.path.expanduser("~")
    Popen(f'echo machine {urs} >> {homeDir + os.sep}{nrc}', shell=True)
    Popen(f'echo login {getpass(prompt=prompts[0])} >> {homeDir + os.sep}{nrc}', shell=True)
    Popen(f'echo password {getpass(prompt=prompts[1])} >> {homeDir + os.sep}{nrc}', shell=True)
    del homeDir
del urs, prompts

In [3]:
# Find last page
base_url = 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/VNP13A2.v002/items/'
collections_response = r.get(base_url, verify=False).json()
total_items = collections_response['numberMatched']
items_returned = collections_response['numberReturned']
last_page = total_items // items_returned
if total_items % items_returned != 0:
    last_page += 1
last_page_url = base_url + f"?page={last_page}"
print(last_page_url)
# https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/VNP13A2.v002/items/?page=11229
last_response = r.get(last_page_url, verify=False).json()

for i in last_response['features']:
    print(i['assets']['browse']['href'])

c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmr.earthdata.nasa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/VNP13A2.v002/items/?page=11286


c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmr.earthdata.nasa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-public/VNP13A2.002/VNP13A2.A2023201.h08v05.002.2023242143159/BROWSE.VNP13A2.A2023201.h08v05.002.2023242143159.1.jpg
https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-public/VNP13A2.002/VNP13A2.A2023201.h21v04.002.2023242143639/BROWSE.VNP13A2.A2023201.h21v04.002.2023242143639.1.jpg
https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-public/VNP13A2.002/VNP13A2.A2023201.h30v12.002.2023242143950/BROWSE.VNP13A2.A2023201.h30v12.002.2023242143950.1.jpg


# Testing logic from code that Justin showed

In [8]:
# Perform a search
lp_search = 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/search'

params = {}

# Set a limit
lim = 100
params['limit'] = lim  # Add in a limit parameter to retrieve 100 items at a time.

# Set the bounding box
field = gp.read_file('Field_Boundary.geojson')
fieldShape = field['geometry'][0]
bbox = '-141.00275,41.6751050889,-52.3231986987,83.6751'
params['bbox'] = bbox

# Choose your collection
collection = "VNP13A2.v002"
params["collections"] = [collection]

# Set the time
date_time = "2023-07-31T00:00:00Z/2023-08-31T23:59:59Z"
params['datetime'] = date_time

# Initialize variables for spatial information and canvas
geoInfo = None  # Initialize spatial information
overall_extent = None  # Initialize overall spatial extent

# Make the request
vnp_items = requests.post(lp_search, json=params, verify=False).json()['features']
print(len(vnp_items))

username = 'infantej2'
password = 'QWEqwe123!@#'

for item_url in vnp_items:
    with requests.Session() as session:
        session.auth = (username, password)
        response = session.get(item_url['assets']['data']['href'], verify=False)
        print(item_url)

        if response.status_code == 200:
            with h5py.File(io.BytesIO(response.content), 'r') as hdf5_file:
                all_h5_objs = []
                hdf5_file.visit(all_h5_objs.append)
                info = hdf5_file['HDFEOS INFORMATION']['StructMetadata.0'][()].split()
                metadata_byte2str = [s.decode('utf-8') for s in info]
                print(f'Metadata: {metadata_byte2str}')
                ulc = [i for i in metadata_byte2str if 'UpperLeftPointMtrs' in i]
                ulcLon = float(ulc[0].replace('=', ',').replace('(', '') \
                        .replace(')', '').split(',')[1])
                ulcLat = float(ulc[0].replace('=', ',').replace('(', '') \
                        .replace(')', '').split(',')[2])
                geoInfo = (ulcLon, 0, ulcLat, 0)
                print(f'Geoinfo: {geoInfo}')

                # Extract NDVI dataset
                ndvi_dataset = hdf5_file['HDFEOS/GRIDS/VIIRS_Grid_16Day_VI_1km/Data Fields/1 km 16 days NDVI'][:]
                print(ndvi_dataset)
                sinu=rio.crs.CRS.from_proj4("+proj=sinu +R=6371007.181 +nadgrids=@null +wktext")
                wgs84=rio.crs.CRS.from_epsg(4326)
                # Update overall spatial extent to cover all datasets
                if overall_extent is None:
                    overall_extent = geoInfo
                else:
                    # Update extent to include current dataset's extent
                    overall_extent = (
                        min(overall_extent[0], geoInfo[0]),
                        min(overall_extent[1], geoInfo[1]),
                        max(overall_extent[2], geoInfo[2]),
                        max(overall_extent[3], geoInfo[3])
                    )
                print(f'Overall extent: {overall_extent}')

# Create an empty canvas to hold the mosaic
width = int((overall_extent[2] - overall_extent[0]) / 1000)  # Adjust the resolution if necessary
height = int((overall_extent[3] - overall_extent[1]) / 1000)  # Adjust the resolution if necessary
canvas = np.zeros((height, width), dtype=np.uint16)

print(f'Overall width: {width}')
print(f'Overall height: {height}')
print(f'The canvas: {canvas}')

"""# Iterate through NDVI datasets and place them on the canvas
for item_url in vnp_items:
    with requests.Session() as session:
        session.auth = (username, password)
        response = session.get(item_url['assets']['data']['href'], verify=False)
        print(item_url)

        if response.status_code == 200:
            with h5py.File(io.BytesIO(response.content), 'r') as hdf5_file:
                # Adjust the following to place the NDVI dataset correctly on the canvas
                # Determine the position on the canvas based on spatial info
                col_start = int((geoInfo[0] - overall_extent[0]) / 1000)
                col_end = col_start + ndvi_dataset.shape[1]
                row_start = int((geoInfo[2] - overall_extent[1]) / 1000)
                row_end = row_start + ndvi_dataset.shape[0]

                # Place the NDVI data on the canvas
                canvas[row_start:row_end, col_start:col_end] = ndvi_dataset

output_tiff = 'mosaic_ndvi.tif'

# Define the transformation (georeferencing) information
transform = from_origin(overall_extent[0], overall_extent[3], 1000, 1000)  # Adjust the resolution if necessary

# Define the CRS 
crs = 'EPSG:4326'  

# Create the GeoTIFF file and write the combined NDVI data
with rio.open(output_tiff, 'w', driver='GTiff', height=canvas.shape[0], width=canvas.shape[1], dtype=canvas.dtype, crs=crs, transform=transform) as dst:
    dst.write(canvas, 1)  # Assuming a single-band GeoTIFF

print("Mosaic GeoTIFF created:", output_tiff)"""


c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmr.earthdata.nasa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


29


c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.lpdaac.earthdatacloud.nasa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'urs.earthdata.nasa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.lpdaac.earthdata

{'type': 'Feature', 'id': 'VNP13A2.A2023201.h14v04.002.2023242143456', 'stac_version': '1.0.0', 'stac_extensions': [], 'collection': 'VNP13A2.v002', 'geometry': {'type': 'Polygon', 'coordinates': [[[-52.1199, 39.7994], [-38.9707, 39.828], [-46.5357, 50.0429], [-62.229, 50], [-52.1199, 39.7994]]]}, 'bbox': [-62.229, 39.7994, -38.9707, 50.287031], 'links': [{'rel': 'self', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/VNP13A2.v002/items/VNP13A2.A2023201.h14v04.002.2023242143456'}, {'rel': 'parent', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/VNP13A2.v002'}, {'rel': 'collection', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/VNP13A2.v002'}, {'rel': 'root', 'href': 'https://cmr.earthdata.nasa.gov/stac/'}, {'rel': 'provider', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD'}, {'rel': 'via', 'href': 'https://cmr.earthdata.nasa.gov/search/concepts/G2758988346-LPCLOUD.json'}, {'rel': 'via', 'href': 'https://cmr.earthdata.nasa.gov/

c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.lpdaac.earthdatacloud.nasa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'urs.earthdata.nasa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.lpdaac.earthdata

...

{'type': 'Feature', 'id': 'VNP13A2.A2023201.h17v01.002.2023242143536', 'stac_version': '1.0.0', 'stac_extensions': [], 'collection': 'VNP13A2.v002', 'geometry': {'type': 'Polygon', 'coordinates': [[[-29.1844, 69.3805], [0.0201, 69.395], [0.1644, 80.0269], [-58.6471, 79.8968], [-29.1844, 69.3805]]]}, 'bbox': [-58.6471, 69.3805, 0.1644, 81.234548], 'links': [{'rel': 'self', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/VNP13A2.v002/items/VNP13A2.A2023201.h17v01.002.2023242143536'}, {'rel': 'parent', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/VNP13A2.v002'}, {'rel': 'collection', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/VNP13A2.v002'}, {'rel': 'root', 'href': 'https://cmr.earthdata.nasa.gov/stac/'}, {'rel': 'provider', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD'}, {'rel': 'via', 'href': 'https://cmr.earthdata.nasa.gov/search/concepts/G2758989526-LPCLOUD.json'}, {'rel': 'via', 'href': 'https://cmr.earthdata.nasa.gov

c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.lpdaac.earthdatacloud.nasa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'urs.earthdata.nasa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.lpdaac.earthdata

{'type': 'Feature', 'id': 'VNP13A2.A2023201.h12v02.002.2023242143402', 'stac_version': '1.0.0', 'stac_extensions': [], 'collection': 'VNP13A2.v002', 'geometry': {'type': 'Polygon', 'coordinates': [[[-124.4071, 59.0509], [-94.8654, 59.2608], [-114.543, 67.6076], [136.0702, 56.5412], [-124.4071, 59.0509]]]}, 'bbox': [136.0702, 56.5412, -94.8654, 73.855573], 'links': [{'rel': 'self', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/VNP13A2.v002/items/VNP13A2.A2023201.h12v02.002.2023242143402'}, {'rel': 'parent', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/VNP13A2.v002'}, {'rel': 'collection', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/VNP13A2.v002'}, {'rel': 'root', 'href': 'https://cmr.earthdata.nasa.gov/stac/'}, {'rel': 'provider', 'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD'}, {'rel': 'via', 'href': 'https://cmr.earthdata.nasa.gov/search/concepts/G2758989534-LPCLOUD.json'}, {'rel': 'via', 'href': 'https://cmr.earthdata

c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.lpdaac.earthdatacloud.nasa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'urs.earthdata.nasa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.lpdaac.earthdata

KeyboardInterrupt: 

# Perform search and read data using satpy

In [16]:
# Perform a search
lp_search = 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/search'

params = {}

# Set a limit
lim = 100
params['limit'] = lim  # Add in a limit parameter to retrieve 100 items at a time.

# Set the bounding box
field = gp.read_file('Field_Boundary.geojson')
fieldShape = field['geometry'][0]
#bbox = f'{fieldShape.bounds[0]},{fieldShape.bounds[1]},{fieldShape.bounds[2]},{fieldShape.bounds[3]}'    # Defined from ROI bounds
bbox = '-141.00275,41.6751050889,-52.3231986987,83.6751'
print(bbox)
params['bbox'] = bbox 

# Choose your collection
collection = "VNP13A2.v002"
params["collections"] = [collection]

# Set the time
date_time = "2023-07-18T00:00:00Z/2023-08-18T23:59:59Z"
params['datetime'] = date_time

# Make the request
vnp_items = r.post(lp_search, json=params, verify=False).json()['features']
print(len(vnp_items))

# Define an empty list to store the dataset arrays
datasets = []

scene = Scene()

# Iterate through vnp_items and read the datasets
for i, item in enumerate(vnp_items):
    item_url = item['assets']['data']['href']
    username = 'infantej2'
    password = 'QWEqwe123!@#'

    with requests.Session() as session:
        session.auth = (username, password)
        response = session.get(item_url, verify=False)
        print(item_url)
        
        if response.status_code == 200:
            
            hdf5_data = hdf5_reader(response.content)
            scene.add(hdf5_data)

            print(f"Processed item {i}")
        else:
            print(f"Authentication failed or the file is inaccessible for item {i}")

scene.show()
        
'''
# Merge the DataArrays using rioxarray's merge
merged_data_array = merge_arrays(datasets)

# Define the output file
output_file = "mosaic_h5.tif"

# Write the merged dataset to the output file
merged_data_array.rio.to_raster(output_file)
'''
print("Mosaic created successfully.")

-141.00275,41.6751050889,-52.3231986987,83.6751


c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmr.earthdata.nasa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


29


NameError: name 'Scene' is not defined

# Perform search and read data using h5py and xarray

In [15]:
# Perform a search
lp_search = 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/search'

params = {}

# Set a limit
lim = 100
params['limit'] = lim  # Add in a limit parameter to retrieve 100 items at a time.

# Set the bounding box
field = gp.read_file('Field_Boundary.geojson')
fieldShape = field['geometry'][0]
#bbox = f'{fieldShape.bounds[0]},{fieldShape.bounds[1]},{fieldShape.bounds[2]},{fieldShape.bounds[3]}'    # Defined from ROI bounds
bbox = '-141.00275,41.6751050889,-52.3231986987,83.6751'
print(bbox)
params['bbox'] = bbox 

# Choose your collection
collection = "VNP13A2.v002"
params["collections"] = [collection]

# Set the time
date_time = "2023-07-18T00:00:00Z/2023-08-18T23:59:59Z"
params['datetime'] = date_time

# Make the request
vnp_items = r.post(lp_search, json=params, verify=False).json()['features']
print(len(vnp_items))

# Define an empty list to store the dataset arrays
datasets = []

# Iterate through vnp_items and read the datasets
for i, item in enumerate(vnp_items):
    item_url = item['assets']['data']['href']
    username = 'infantej2'
    password = 'QWEqwe123!@#'

    with requests.Session() as session:
        session.auth = (username, password)
        response = session.get(item_url, verify=False)
        print(item_url)
        
        if response.status_code == 200:
            # Open the HDF5 file from the response content
            with h5py.File(io.BytesIO(response.content), 'r') as hdf5_file:
                dataset = hdf5_file['HDFEOS/GRIDS/VIIRS_Grid_16Day_VI_1km/Data Fields/1 km 16 days NDVI'][:]

                # Convert the dataset to an xarray DataArray
                data_array = xr.DataArray(dataset, dims=("y", "x"))
                print(dataset)
                print(data_array)

                # Append the DataArray to list
                datasets.append(data_array)

            print(f"Processed item {i}")
        else:
            print(f"Authentication failed or the file is inaccessible for item {i}")
        
'''
# Merge the DataArrays using rioxarray's merge
merged_data_array = merge_arrays(datasets)

# Define the output file
output_file = "mosaic_h5.tif"

# Write the merged dataset to the output file
merged_data_array.rio.to_raster(output_file)
'''
print("Mosaic created successfully.")

-141.00275,41.6751050889,-52.3231986987,83.6751


c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmr.earthdata.nasa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


29


c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.lpdaac.earthdatacloud.nasa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'urs.earthdata.nasa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.lpdaac.earthdata

https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/VNP13A2.002/VNP13A2.A2023185.h08v04.002.2023201103423/VNP13A2.A2023185.h08v04.002.2023201103423.h5
[[-15000 -15000 -15000 ... -15000 -15000 -15000]
 [-15000 -15000 -15000 ... -15000 -15000 -15000]
 [-15000 -15000 -15000 ... -15000 -15000 -15000]
 ...
 [-15000 -15000 -15000 ...   2886   3018   2916]
 [-15000 -15000 -15000 ...   2996   2759   2552]
 [-15000 -15000 -15000 ...   3121   2678   2321]]
<xarray.DataArray (y: 1200, x: 1200)>
array([[-15000, -15000, -15000, ..., -15000, -15000, -15000],
       [-15000, -15000, -15000, ..., -15000, -15000, -15000],
       [-15000, -15000, -15000, ..., -15000, -15000, -15000],
       ...,
       [-15000, -15000, -15000, ...,   2886,   3018,   2916],
       [-15000, -15000, -15000, ...,   2996,   2759,   2552],
       [-15000, -15000, -15000, ...,   3121,   2678,   2321]], dtype=int16)
Dimensions without coordinates: y, x
Processed item 0


...

https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/VNP13A2.002/VNP13A2.A2023185.h15v01.002.2023201122420/VNP13A2.A2023185.h15v01.002.2023201122420.h5
[[-15000 -15000 -15000 ... -15000 -15000 -15000]
 [-15000 -15000 -15000 ... -15000 -15000 -15000]
 [-15000 -15000 -15000 ... -15000 -15000 -15000]
 ...
 [-15000 -15000 -15000 ... -15000 -15000 -15000]
 [-15000 -15000 -15000 ... -15000 -15000 -15000]
 [-15000 -15000 -15000 ... -15000 -15000 -15000]]
<xarray.DataArray (y: 1200, x: 1200)>
array([[-15000, -15000, -15000, ..., -15000, -15000, -15000],
       [-15000, -15000, -15000, ..., -15000, -15000, -15000],
       [-15000, -15000, -15000, ..., -15000, -15000, -15000],
       ...,
       [-15000, -15000, -15000, ..., -15000, -15000, -15000],
       [-15000, -15000, -15000, ..., -15000, -15000, -15000],
       [-15000, -15000, -15000, ..., -15000, -15000, -15000]], dtype=int16)
Dimensions without coordinates: y, x
Processed item 26


c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.lpdaac.earthdatacloud.nasa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'urs.earthdata.nasa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.lpdaac.earthdata

https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/VNP13A2.002/VNP13A2.A2023185.h20v01.002.2023201124919/VNP13A2.A2023185.h20v01.002.2023201124919.h5
[[-15000 -15000 -15000 ... -15000 -15000 -15000]
 [-15000 -15000 -15000 ... -15000 -15000 -15000]
 [-15000 -15000 -15000 ... -15000 -15000 -15000]
 ...
 [-15000 -15000 -15000 ... -15000 -15000 -15000]
 [-15000 -15000 -15000 ... -15000 -15000 -15000]
 [-15000 -15000 -15000 ... -15000 -15000 -15000]]
<xarray.DataArray (y: 1200, x: 1200)>
array([[-15000, -15000, -15000, ..., -15000, -15000, -15000],
       [-15000, -15000, -15000, ..., -15000, -15000, -15000],
       [-15000, -15000, -15000, ..., -15000, -15000, -15000],
       ...,
       [-15000, -15000, -15000, ..., -15000, -15000, -15000],
       [-15000, -15000, -15000, ..., -15000, -15000, -15000],
       [-15000, -15000, -15000, ..., -15000, -15000, -15000]], dtype=int16)
Dimensions without coordinates: y, x
Processed item 27


c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.lpdaac.earthdatacloud.nasa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'urs.earthdata.nasa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\cinfante\OneDrive - NRCan RNCan\Documents\Greenup\NASA-LPDAAC-Testing\venv\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.lpdaac.earthdata

https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/VNP13A2.002/VNP13A2.A2023185.h11v02.002.2023201114934/VNP13A2.A2023185.h11v02.002.2023201114934.h5
[[-15000 -15000 -15000 ... -15000 -15000 -15000]
 [-15000 -15000 -15000 ... -15000 -15000 -15000]
 [-15000 -15000 -15000 ... -15000 -15000 -15000]
 ...
 [  3993   2320   1225 ...   6449   1600   1750]
 [  2026   5086    798 ...   6305   6458   6333]
 [  2380   1372    868 ...   6075   6073   5961]]
<xarray.DataArray (y: 1200, x: 1200)>
array([[-15000, -15000, -15000, ..., -15000, -15000, -15000],
       [-15000, -15000, -15000, ..., -15000, -15000, -15000],
       [-15000, -15000, -15000, ..., -15000, -15000, -15000],
       ...,
       [  3993,   2320,   1225, ...,   6449,   1600,   1750],
       [  2026,   5086,    798, ...,   6305,   6458,   6333],
       [  2380,   1372,    868, ...,   6075,   6073,   5961]], dtype=int16)
Dimensions without coordinates: y, x
Processed item 28
Mosaic created successfully.


In [ ]:
# Subset by Band

v = vnp_items[20]

evi_band_links = []

# Define which HLS product is being accessed
if v['assets']['browse']['href'].split('/')[4] == 'VNP13A2.v002':
    evi_bands = ['B8A', 'B04', 'B02', 'Fmask'] # NIR RED BLUE Quality for S30
else:
    evi_bands = ['B05', 'B04', 'B02', 'Fmask'] # NIR RED BLUE Quality for L30

# Subset the assets in the item down to only the desired bands
for a in v['assets']: 
    if any(b == a for b in evi_bands):
        evi_band_links.append(v['assets'][a]['href'])
for e in evi_band_links: print(e)

image = io.imread(v['assets']['browse']['href'])  # Load jpg browse image into memory

# Basic plot of the image
plt.figure(figsize=(10,10))              
plt.imshow(image)
plt.show()

del image # Remove the browse image

In [ ]:
# Use vsicurl to load the data directly into memory (be patient, may take a few seconds)
for e in evi_band_links:
    print(e)
    if e.rsplit('.', 2)[-2] == evi_bands[0]:      # NIR index
        nir = rio.open(e)
    elif e.rsplit('.', 2)[-2] == evi_bands[1]:    # red index
        red = rio.open(e)
    elif e.rsplit('.', 2)[-2] == evi_bands[2]:    # blue index
        blue = rio.open(e)
    elif e.rsplit('.', 2)[-2] == evi_bands[3]:    # Fmask index
        fmask = rio.open(e)
print("The COGs have been loaded into memory!")

In [ ]:
geo_CRS = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)  # Source coordinate system of the ROI
utm = pyproj.Proj(nir.crs)                                                  # Destination coordinate system
project = pyproj.Transformer.from_proj(geo_CRS, utm)                        # Set up the transformation
fsUTM = transform(project.transform, fieldShape)                            # Apply reprojection